In [2]:
import tensorflow as tf
import os

In [3]:
tf.__version__

'2.0.0-alpha0'

In [4]:
data = tf.keras.utils.get_file(origin='https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
                                         fname='flower_photos', untar=True)

In [5]:
label_names = [x for x in os.listdir(data) if not x[-4:] == ".txt"]
label_name_index = dict((name, index) for index,name in enumerate(label_names))

In [16]:
image_path_list = []
image_label_list = []
for label in label_names:
    path = os.path.join(data,label)
    for x in os.listdir(path):
        image_path_list.append(os.path.join(path,x))
        image_label_list.append(label_name_index[label])

In [17]:
ds = tf.data.Dataset.from_tensor_slices((image_path_list, image_label_list))


In [18]:
def load_and_preprocess_from_path_label(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [192, 192])
    image /= 255.0 
    image = 2*image-1
    return image, label

In [19]:
ds = ds.apply( tf.data.experimental.shuffle_and_repeat(buffer_size=len(image_path_list)))
ds = ds.apply(tf.data.experimental.map_and_batch(map_func=load_and_preprocess_from_path_label, batch_size=32))
ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [23]:
ds
# image_label_ds = ds.map(load_and_preprocess_from_path_label, num_parallel_calls=tf.data.experimental.AUTOTUNE)

<PrefetchDataset shapes: ((None, 192, 192, 3), (None,)), types: (tf.float32, tf.int32)>

Now the data pipeline is ready. We can feed this pipeline to train the model

In [25]:
mobile_net = tf.keras.applications.MobileNetV2(input_shape=(192, 192, 3), include_top=False)
mobile_net.trainable=False

In [53]:
image_batch, label_batch = next(iter(ds))

In [ ]:
feature_map_batch = mobile_net(image_batch)
print(feature_map_batch.shape)

In [26]:
model = tf.keras.Sequential([
  mobile_net,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(len(label_names))])

In [27]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=["accuracy"])

In [28]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_192 (Model) (None, 6, 6, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 6405      
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


In [34]:
model.fit(ds,epochs=1, steps_per_epoch= tf.math.ceil(len(image_path_list)/32).numpy())

  4/115 [>.............................] - ETA: 3:00 - loss: 1.6094 - accuracy: 0.3047

KeyboardInterrupt: 